In [24]:
import os
from tqdm import tqdm
from typing import List, Optional

import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport

pd.set_option("display.max_columns", 300)

ModuleNotFoundError: No module named 'pandas_profiling'

In [25]:
!pip install pandas_profiling

     |████████████████████████████████| 258 kB 535 kB/s eta 0:00:01
     |████████████████████████████████| 64 kB 2.2 MB/s eta 0:00:011
     |████████████████████████████████| 10.5 MB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 3.9 MB/s  eta 0:00:01
     |████████████████████████████████| 69 kB 8.8 MB/s  eta 0:00:01
     |████████████████████████████████| 283 kB 18.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 14.0 MB/s eta 0:00:01
     |████████████████████████████████| 125 kB 17.1 MB/s eta 0:00:01
     |████████████████████████████████| 11.6 MB 15.1 MB/s eta 0:00:01
  Using cached requests-2.24.0-py2.py3-none-any.whl (61 kB)
     |████████████████████████████████| 599 kB 21.8 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 5.7 MB/s  eta 0:00:01
     |████████████████████████████████| 1.6 MB 42.0 MB/s eta 0:00:01
     |████████████████████████████████| 291 kB 23.0 MB/s eta 0:00:01
  Using cached certifi-2020.6.20-py2.py3-none-a

  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27084 sha256=f09fdde4bb1cd168610f2df3393cbe2793f08c11441b304d5d4ea73b9d673925
  Stored in directory: /home/bocharick/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
  Created wheel for imagehash: filename=ImageHash-4.1.0-py2.py3-none-any.whl size=291991 sha256=63855a16d6757dff0d8ea79b7b02d48311fb9a49ca130cfb7a048ecbc22d7051
  Stored in directory: /home/bocharick/.cache/pip/wheels/16/72/b0/e5be34699908d9ff25dcb3debbb717987b766af61bbddffdfe
Successfully built htmlmin imagehash
  Attempting uninstall: pandas
    Found existing installation: pandas 0.25.1
    Uninstalling pandas-0.25.1:
      Successfully uninstalled pandas-0.25.1
  Attempting uninstall: networkx
    Found existing installation: networkx 2.3
    Uninstalling networkx-2.3:
      Successfully uninstalled networkx-2.3
  Attempting uninstall: attrs
    Found existing installation: attrs 19.2.0
    Uninstalling attrs-19.2.0:

In [2]:
os.listdir("data")

['train.csv',
 'applications_history.csv',
 'sample_submit.csv',
 'payments.csv',
 'client_profile.csv',
 'test.csv',
 'bki.csv']

# TRAIN.CSV

In [3]:
train = pd.read_csv("data/train.csv")
train.head(n=2)

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE
0,123687442,0,Cash
1,123597908,1,Cash


#### APPLICATION_NUMBER
номер клиента, либо номер кредитной заявки

#### TARGET
целевая переменная, то что мы хотим спрогнозировать

#### NAME_CONTRACT_TYPE
тип контракта. какой кредитный продукт хочет клиент взять.


In [4]:
train["NAME_CONTRACT_TYPE"].value_counts()

Cash           99551
Credit Card    10542
Name: NAME_CONTRACT_TYPE, dtype: int64

# SAMPLE_SUBMIT.CSV

In [5]:
sample = pd.read_csv("data/sample_submit.csv")
sample.head(n=5)

,APPLICATION_NUMBER,TARGET
0,123724268,0
1,123456549,0
2,123428178,0
3,123619984,0
4,123671104,0


#### APPLICATION_NUMBER
номер клиента, либо номер кредитной заявки

#### TARGET
целевая переменная, то что мы хотим спрогнозировать

# CLIENT_PROFILE.CSV

In [6]:
client_profile = pd.read_csv("data/client_profile.csv")
client_profile.head(n=2)

,APPLICATION_NUMBER,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,123666076,F,0,157500.0,270000.0,13500.0,Incomplete higher,Civil marriage,0.008068,8560,1549,NaN,1,0,2.0,0.329471,0.236315,0.678568,0.0,0.0,0.0,0.0,1.0,2.0
1,123423688,F,0,270000.0,536917.5,28467.0,Secondary / secondary special,Married,0.020246,23187,365243,NaN,0,0,2.0,NaN,0.442295,0.802745,0.0,0.0,0.0,0.0,1.0,1.0


In [7]:
# print(*client_profile.columns, sep="\n")

#### APPLICATION_NUMBER
номер клиента, id клиента, для связи

#### GENDER
пол

#### CHILDRENS
число детей

#### TOTAL_SALARY
зарплата

#### AMOUNT_CREDIT
сумма кредита которую хочет взять

#### AMOUNT_ANNUITY
сумма ежемесячного платежа

#### EDUCATION_LEVEL
уровень образования

#### FAMILY_STATUS
семейное положение

#### REGION_POPULATION
население региона, нормализованное на какую-то величину

#### AGE
возраст, в днях

#### DAYS_ON_LAST_JOB
продолжительность работы на последнем рабочем месте, в днях

#### OWN_CAR_AGE
флаг?? наличия автомобиля

#### FLAG_PHONE
флаг подтвержденного номера телефона

#### FLAG_EMAIL
флаг подтвержденного емэйла

#### FAMILY_SIZE
размер семьи

#### EXTERNAL_SCORING_RATING_1

таргет скоринг (вероятность дефолта), оцененный другой компанией 1

#### EXTERNAL_SCORING_RATING_2
таргет скоринг (вероятность дефолта), оцененный другой компанией 2

#### EXTERNAL_SCORING_RATING_3
таргет скоринг (вероятность дефолта), оцененный другой компанией 3

#### AMT_REQ_CREDIT_BUREAU_HOUR
количество запросов сделанное в бки по данному клиенту за последний час

#### AMT_REQ_CREDIT_BUREAU_DAY
количество запросов сделанное в бки по данному клиенту за последний день

#### AMT_REQ_CREDIT_BUREAU_WEEK
количество запросов сделанное в бки по данному клиенту за последнюю неделю

#### AMT_REQ_CREDIT_BUREAU_MON
количество запросов сделанное в бки по данному клиенту за последний месяц

#### AMT_REQ_CREDIT_BUREAU_QRT
количество запросов сделанное в бки по данному клиенту за последний квартал

#### AMT_REQ_CREDIT_BUREAU_YEAR
количество запросов сделанное в бки по данному клиенту за последний год


In [8]:
client_profile["OWN_CAR_AGE"].value_counts()

7.0     6051
6.0     5199
3.0     5189
8.0     4845
2.0     4771
4.0     4525
1.0     4313
9.0     4061
10.0    3885
14.0    3721
13.0    3704
12.0    3426
11.0    3393
5.0     2906
15.0    2901
16.0    2736
17.0    2353
18.0    1969
64.0    1966
0.0     1752
19.0    1526
20.0    1233
21.0    1188
22.0    1000
24.0     929
23.0     885
65.0     711
25.0     693
26.0     461
28.0     438
27.0     406
29.0     339
30.0     267
31.0     228
32.0     170
34.0     146
35.0     124
33.0     108
36.0      97
38.0      74
40.0      70
39.0      65
37.0      59
41.0      43
42.0      39
44.0      19
43.0      15
54.0      12
45.0      10
49.0       5
55.0       4
51.0       3
46.0       3
69.0       1
56.0       1
57.0       1
47.0       1
52.0       1
Name: OWN_CAR_AGE, dtype: int64

# APPLICATIONS_HISTORY.CSV

In [9]:
apps = pd.read_csv("data/applications_history.csv")
apps.head(n=2)

,PREV_APPLICATION_NUMBER,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,AMOUNT_ANNUITY,AMT_APPLICATION,AMOUNT_CREDIT,AMOUNT_PAYMENT,AMOUNT_GOODS_PAYMENT,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,SELLERPLACE_AREA,CNT_PAYMENT,NAME_YIELD_GROUP,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,49298709,123595216,NaN,1730.430,17145.0,17145.0,0.0,17145.0,Approved,73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,35,12.0,middle,365243.0,42.0,300.0,42.0,37.0,0.0
1,50070639,123431468,Cash,25188.615,607500.0,679671.0,NaN,607500.0,Approved,164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,-1,36.0,low_action,365243.0,134.0,916.0,365243.0,365243.0,1.0


#### PREV_APPLICATION_NUMBER
предыдущие заявки

#### APPLICATION_NUMBER
ключ, id клиента для связи

#### NAME_CONTRACT_TYPE
тип контракта, тоже самое что в train.csv

#### AMOUNT_ANNUITY
сумма ежемесячного платежа по предыдущему кредиту

#### AMT_APPLICATION
сумма кредита, которую он указал в заявке

#### AMOUNT_CREDIT
сумма кредита которую ему дали по факту

#### AMOUNT_PAYMENT


#### AMOUNT_GOODS_PAYMENT
стоимость товара, если это кредит на товар

#### NAME_CONTRACT_STATUS
статус заявки

#### DAYS_DECISION
сколько дней назад было принято решение по этой заявке

#### NAME_PAYMENT_TYPE
каким образом платеж будет проведен

#### CODE_REJECT_REASON
если заявка была отказная, то по ней причина отказа

#### NAME_TYPE_SUITE
каким образом заявка пришла, то есть канал привлечения

#### NAME_CLIENT_TYPE
тип клиента, уже был в базе или новый

#### NAME_GOODS_CATEGORY
категория товара, который был куплен

#### NAME_PORTFOLIO
подпродукт

#### NAME_PRODUCT_TYPE
подпродукт

#### SELLERPLACE_AREA
локация где была совершена продажа, локация офиса либо торговой точки

#### CNT_PAYMENT
количество платежей по кредиту

#### NAME_YIELD_GROUP
некоторая сегментация клиента по какому-то признаку. банки сегментируют клиентов и разным сегментам могут делать разные предложения. что-то типо кластеризации

#### DAYS_FIRST_DRAWING


#### DAYS_FIRST_DUE
сколько дней до первого платежа

#### DAYS_LAST_DUE_1ST_VERSION
сколько дней после первого платежа

#### DAYS_LAST_DUE
сколько дней после последнего платежа

#### DAYS_TERMINATION
(_ВОЗМОЖНО_) дней когда кредит был закрыт

#### NFLAG_INSURED_ON_APPROVAL
флаг, есть страховка по данному кредиту или нет


# BKI.CSV

In [10]:
bki = pd.read_csv("data/bki.csv")
bki.head(n=2)

,APPLICATION_NUMBER,BUREAU_ID,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,123538884,5223613,Active,currency 1,718.0,0,377.0,NaN,19386.81,0,675000.00,320265.495,0.0,0.0,Consumer credit,39.0,NaN
1,123436670,6207544,Closed,currency 1,696.0,0,511.0,511.0,0.00,0,93111.66,0.000,0.0,0.0,Consumer credit,505.0,NaN


In [11]:
# print(*bki.columns, sep="\n")

#### APPLICATION_NUMBER
номер заявки, ключ

#### BUREAU_ID
какое бюро кредитных историй было опрошено

#### CREDIT_ACTIVE
статус кредита

#### CREDIT_CURRENCY
валюта, в которой был взят кредит

#### DAYS_CREDIT
какое время назад был взят кредит

#### CREDIT_DAY_OVERDUE
сколько дней просрочки

#### DAYS_CREDIT_ENDDATE
сколько дней до закрытия кредита

#### DAYS_ENDDATE_FACT
сколько дней до закрытия (тут с фактическим и планируемым мутки, типо если где-то был изменен график кредита)

#### AMT_CREDIT_MAX_OVERDUE
максимальная сумма просрочки

#### CNT_CREDIT_PROLONG
был ли пролонгирован договор

#### AMT_CREDIT_SUM
сумма кредита

#### AMT_CREDIT_SUM_DEBT
сумма средств которая была внесена

#### AMT_CREDIT_SUM_LIMIT
кредитный лимит, актуален для кредитных карт

#### AMT_CREDIT_SUM_OVERDUE
общая сумма просрочки

#### CREDIT_TYPE
тип кредита

#### DAYS_CREDIT_UPDATE
сколько дней назад было обновление информации по данному кредиту (чем более свежие данные тем больше веры им)

#### AMT_ANNUITY
сумма ежемесячного платежа


# PAYMENTS.CSV

In [12]:
payments = pd.read_csv("data/payments.csv")
payments.head(n=2)

,PREV_APPLICATION_NUMBER,APPLICATION_NUMBER,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,49011181,123664960,1.0,5,1002.0,1015.0,12156.615,12156.615
1,48683432,123497205,1.0,13,442.0,432.0,18392.535,10047.645


In [13]:
# print(*payments.columns, sep="\n")

#### PREV_APPLICATION_NUMBER
предыдущая заявка

#### APPLICATION_NUMBER
текущая заявка

#### NUM_INSTALMENT_VERSION


#### NUM_INSTALMENT_NUMBER
номер платежа

#### DAYS_INSTALMENT
сколько дней назад должен был этот платеж относительно даты подачи заявки

#### DAYS_ENTRY_PAYMENT
когда был совершен фактический платеж

#### AMT_INSTALMENT
сумма платежа по графику

#### AMT_PAYMENT
сколько клиент заплатил фактически


# TEST.CSV

In [14]:
test = pd.read_csv("data/test.csv")
test.head(n=2)

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE
0,123724268,Cash
1,123456549,Cash


In [15]:
#########################################################
###########################################################

In [16]:
123687442

123687442

In [17]:
train.loc[train.APPLICATION_NUMBER == 123687442]

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE
0,123687442,0,Cash


In [18]:
client_profile.loc[client_profile.APPLICATION_NUMBER == 123687442]
# все по одному разу

,APPLICATION_NUMBER,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
67474,123687442,M,1,157500.0,855000.0,25128.0,Secondary / secondary special,Married,0.019101,15728,1719,11.0,0,0,3.0,0.700784,0.645914,0.71657,0.0,0.0,1.0,0.0,0.0,2.0


In [19]:
apps.loc[apps.APPLICATION_NUMBER == 123687442]

,PREV_APPLICATION_NUMBER,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,AMOUNT_ANNUITY,AMT_APPLICATION,AMOUNT_CREDIT,AMOUNT_PAYMENT,AMOUNT_GOODS_PAYMENT,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,SELLERPLACE_AREA,CNT_PAYMENT,NAME_YIELD_GROUP,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
239762,49915814,123687442,NaN,6510.015,71668.80,64500.3,7168.50,71668.80,Approved,240,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,20,12.0,middle,365243.0,143.0,187.0,365243.0,365243.0,0.0
509872,49713743,123687442,NaN,9262.215,72634.14,70758.0,7267.14,72634.14,Approved,2702,Non-cash from your account,XAP,Family,Repeater,Photo / Cinema Equipment,POS,XNA,90,10.0,high,365243.0,2671.0,2401.0,2401.0,2368.0,1.0
626049,49704707,123687442,NaN,7339.185,62058.60,67518.0,3.60,62058.60,Approved,721,Cash through the bank,XAP,NaN,Refreshed,Computers,POS,XNA,150,10.0,low_action,365243.0,690.0,420.0,420.0,415.0,0.0


In [33]:
apps.loc[apps.DAYS_TERMINATION <10]

,PREV_APPLICATION_NUMBER,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,AMOUNT_ANNUITY,AMT_APPLICATION,AMOUNT_CREDIT,AMOUNT_PAYMENT,AMOUNT_GOODS_PAYMENT,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,SELLERPLACE_AREA,CNT_PAYMENT,NAME_YIELD_GROUP,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
25,48811345,123599046,Cash,22619.520,229500.00,241920.0,NaN,229500.00,Approved,370,Cash through the bank,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,-1,12.0,low_normal,365243.0,340.0,10.0,10.0,3.0,1.0
142,49165885,123539497,NaN,8191.485,76275.00,101682.0,0.00,76275.00,Approved,554,Cash through the bank,XAP,NaN,Repeater,Consumer Electronics,POS,XNA,1500,18.0,high,365243.0,523.0,13.0,13.0,6.0,0.0
162,48598333,123595859,NaN,11493.990,51210.00,55264.5,0.00,51210.00,Approved,157,Cash through the bank,XAP,Other_B,Repeater,Mobile,POS,XNA,120,6.0,high,365243.0,127.0,23.0,7.0,2.0,1.0
296,50037684,123592803,NaN,6873.390,61866.00,69214.5,0.00,61866.00,Approved,220,Cash through the bank,XAP,NaN,Repeater,Audio/Video,POS,XNA,1800,12.0,low_normal,365243.0,190.0,140.0,10.0,5.0,1.0
479,50073446,123654275,NaN,10215.855,105336.00,116460.0,0.00,105336.00,Approved,373,Cash through the bank,XAP,NaN,New,Computers,POS,XNA,4000,12.0,low_action,365243.0,342.0,12.0,12.0,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667089,48829090,123586442,NaN,3853.755,47726.55,38182.5,9544.05,47726.55,Approved,80,Cash through the bank,XAP,NaN,XNA,Consumer Electronics,POS,XNA,70,12.0,middle,365243.0,42.0,288.0,12.0,4.0,0.0
1667097,48721195,123538676,NaN,4393.215,39960.00,39960.0,0.00,39960.00,Approved,397,XNA,XAP,NaN,Refreshed,Mobile,POS,XNA,14,14.0,high,365243.0,342.0,48.0,12.0,6.0,0.0
1667123,49394539,123454319,NaN,6576.705,60750.00,60750.0,0.00,60750.00,Approved,309,Cash through the bank,XAP,Unaccompanied,Repeater,Consumer Electronics,POS,XNA,814,12.0,middle,365243.0,279.0,51.0,9.0,7.0,0.0
1668104,49153132,123647280,NaN,15375.645,288972.00,339493.5,0.00,288972.00,Approved,703,Cash through the bank,XAP,Family,Refreshed,Consumer Electronics,POS,XNA,300,24.0,low_action,365243.0,672.0,18.0,12.0,5.0,0.0


In [34]:
apps.DAYS_TERMINATION.value_counts()

365243.0    225913
233.0          786
184.0          770
170.0          770
163.0          769
             ...  
2795.0           1
2797.0           1
2809.0           1
2808.0           1
2852.0           1
Name: DAYS_TERMINATION, Length: 2830, dtype: int64

In [35]:
365243//365

1000

In [20]:
bki.loc[bki.APPLICATION_NUMBER == 123687442]

,APPLICATION_NUMBER,BUREAU_ID,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
585177,123687442,5640206,Closed,currency 1,2562.0,0,2197.0,2162.0,4714.425,0,56362.5,0.0,0.0,0.0,Consumer credit,2162.0,NaN


In [29]:
bki.loc[bki.APPLICATION_NUMBER == 123687442]

,APPLICATION_NUMBER,BUREAU_ID,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
585177,123687442,5640206,Closed,currency 1,2562.0,0,2197.0,2162.0,4714.425,0,56362.5,0.0,0.0,0.0,Consumer credit,2162.0,NaN


In [21]:
payments.loc[payments.APPLICATION_NUMBER == 123687442]

,PREV_APPLICATION_NUMBER,APPLICATION_NUMBER,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
65196,49796018,123687442,1.0,4,2830.0,2830.0,3289.635,3289.635
79322,49796018,123687442,1.0,7,2740.0,2740.0,3289.635,3289.635
693273,49713743,123687442,1.0,5,2551.0,2547.0,9262.215,9251.865
726215,49713743,123687442,1.0,7,2491.0,2519.0,9262.215,10.350
